In [ ]:


### ============================================
 #PGE1 AI Clinic - Educational Data Analysis
 #End-to-end data cleaning + analysis notebook
 #PERSON: Kothapally SHIVESH REDDY
 #============================================
#
 #---------- Setup ----------
import pandas as pd
import re


df = pd.read_csv("EdStatsData.csv", low_memory=False)

# Standardize column names
df.columns = (
    df.columns
    .str.strip()
    .str.replace("\ufeff", "", regex=False)
    .str.lower()
    .str.replace(r"[\\s/]+", "_", regex=True)
    .str.replace(r"[^a-z0-9_]", "", regex=True)
)

#  Remove empty columns and duplicate rows
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)

#  Trim whitespace in text columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype(str).str.strip()


year_cols = [c for c in df.columns if c.isdigit()]
for col in year_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

#  Drop rows missing critical keys
if 'country_code' in df.columns and 'indicator_code' in df.columns:
    df = df.dropna(subset=['country_code', 'indicator_code'])

#  Drop rows completely empty across all year columns
df = df.dropna(how='all', subset=year_cols)

# Fill missing text fields with 'Unknown'
for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna('Unknown', inplace=True)

# Fill numeric columns with 0
for col in year_cols:
    df[col].fillna(0, inplace=True)


summary = {
    "total_rows": len(df),
    "total_columns": len(df.columns),
    "missing_values": df.isnull().sum().sum(),
    "year_range": (min(map(int, year_cols)), max(map(int, year_cols))) if year_cols else None,
    "top_indicators": df['indicator_name'].value_counts().head(5).to_dict() if 'indicator_name' in df.columns else {}
}

print(" Cleaning complete!")
print("Summary:")
for k, v in summary.items():
    print(f"{k}: {v}")

df.to_csv("cleaned_EdStatsData.csv", index=False)
print("Cleaned file saved as cleaned_EdStatsData.csv")


 Cleaning complete!
Summary:
total_rows: 357405
total_columns: 69
missing_values: 0
year_range: (1970, 2100)
top_indicators: {}
Cleaned file saved as cleaned_EdStatsData.csv


In [ ]:

import pandas as pd

#  Helpers in my project 
def standardize_columns(df):
    df.columns = (
        df.columns.str.strip()
        .str.replace("\ufeff", "", regex=False)
        .str.lower()
        .str.replace(r"[\s/]+", "_", regex=True)
        .str.replace(r"[^a-z0-9_]", "", regex=True)
    )
    return df

def rename_to(df, target_name, candidates):
    """Rename the first existing candidate column to target_name; return df and the resolved name or None."""
    for c in candidates:
        if c in df.columns:
            if c != target_name:
                df = df.rename(columns={c: target_name})
            return df, target_name
    return df, None

df = pd.read_csv("EdStatsSeries.csv", low_memory=False)

# Clean basics 
df = standardize_columns(df)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)

# Trim whitespace
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].astype(str).str.strip()

# Ensure critical key: indicator_code (from series_code or similar)
df, key = rename_to(df, "indicator_code", ["indicator_code", "series_code", "seriescode", "code"])

# Drop rows with missing critical key 
if key:
    before = len(df)
    df = df.dropna(subset=[key])
    print(f"Dropped {before - len(df)} rows missing '{key}'.")

# --- Fill NaNs in non-critical text columns ---
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].fillna("Unknown")

#  Save
df.to_csv("cleaned_EdStatsSeries.csv", index=False)
print(" cleaned_EdStatsSeries.csv saved")


Dropped 0 rows missing 'indicator_code'.
 cleaned_EdStatsSeries.csv saved


In [10]:

import pandas as pd

def standardize_columns(df):
    df.columns = (
        df.columns.str.strip()
        .str.replace("\ufeff", "", regex=False)
        .str.lower()
        .str.replace(r"[\s/]+", "_", regex=True)
        .str.replace(r"[^a-z0-9_]", "", regex=True)
    )
    return df

def rename_to(df, target_name, candidates):
    for c in candidates:
        if c in df.columns:
            if c != target_name:
                df = df.rename(columns={c: target_name})
            return df, target_name
    return df, None

df = pd.read_csv("EdStatsCountry.csv", low_memory=False)

df = standardize_columns(df)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)

for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].astype(str).str.strip()

# Ensure critical key: country_code
df, key = rename_to(df, "country_code", ["country_code", "countrycode", "code"])

if key:
    before = len(df)
    df = df.dropna(subset=[key])
    print(f"Dropped {before - len(df)} rows missing '{key}'.")

# Fill NaNs in non-critical text columns
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].fillna("Unknown")

df.to_csv("cleaned_EdStatsCountry.csv", index=False)
print(" cleaned_EdStatsCountry.csv saved")


Dropped 0 rows missing 'country_code'.
 cleaned_EdStatsCountry.csv saved


In [11]:

import pandas as pd

def standardize_columns(df):
    df.columns = (
        df.columns.str.strip()
        .str.replace("\ufeff", "", regex=False)
        .str.lower()
        .str.replace(r"[\s/]+", "_", regex=True)
        .str.replace(r"[^a-z0-9_]", "", regex=True)
    )
    return df

def rename_to(df, target_name, candidates):
    for c in candidates:
        if c in df.columns:
            if c != target_name:
                df = df.rename(columns={c: target_name})
            return df, target_name
    return df, None

df = pd.read_csv("EdStatsCountry-Series.csv", low_memory=False)

df = standardize_columns(df)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)

for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].astype(str).str.strip()

# Ensure keys: country_code & indicator_code
df, cc = rename_to(df, "country_code", ["country_code", "countrycode", "code"])
df, ic = rename_to(df, "indicator_code", ["indicator_code", "series_code", "seriescode", "code"])

# Drop rows missing either key (only if they exist)
subset_keys = [k for k in [cc, ic] if k]
if subset_keys:
    before = len(df)
    df = df.dropna(subset=subset_keys)
    print(f"Dropped {before - len(df)} rows missing {subset_keys}.")

# Fill NaNs in non-critical text columns
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].fillna("Unknown")

df.to_csv("cleaned_EdStatsCountrySeries.csv", index=False)
print(" cleaned_EdStatsCountrySeries.csv saved")


Dropped 0 rows missing ['country_code', 'indicator_code'].
 cleaned_EdStatsCountrySeries.csv saved


In [12]:

import pandas as pd
import re

def standardize_columns(df):
    df.columns = (
        df.columns.str.strip()
        .str.replace("\ufeff", "", regex=False)
        .str.lower()
        .str.replace(r"[\s/]+", "_", regex=True)
        .str.replace(r"[^a-z0-9_]", "", regex=True)
    )
    return df

def rename_to(df, target_name, candidates):
    for c in candidates:
        if c in df.columns:
            if c != target_name:
                df = df.rename(columns={c: target_name})
            return df, target_name
    return df, None

df = pd.read_csv("EdStatsFootNote.csv", low_memory=False)

df = standardize_columns(df)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)

for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].astype(str).str.strip()

# Ensure keys
df, cc = rename_to(df, "country_code", ["country_code", "countrycode", "code"])
df, ic = rename_to(df, "indicator_code", ["indicator_code", "series_code", "seriescode", "code"])

# Parse 'year' labels like 'YR2001' -> 2001 (if a year-like column exists)
# After standardization, the column is usually 'year'
if "year" in df.columns:
    def parse_year(x):
        s = str(x)
        m = re.search(r"(\d{4})", s)
        return int(m.group(1)) if m else pd.NA
    df["year"] = df["year"].apply(lambda x: parse_year(x))

# Drop rows missing critical keys (country_code, indicator_code)
subset_keys = [k for k in [cc, ic] if k]
if subset_keys:
    before = len(df)
    df = df.dropna(subset=subset_keys)
    print(f"Dropped {before - len(df)} rows missing {subset_keys}.")

# Fill NaNs in non-critical text columns (e.g., description/notes)
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].fillna("Unknown")

df.to_csv("cleaned_EdStatsFootNote.csv", index=False)
print(" cleaned_EdStatsFootNote.csv saved")


Dropped 0 rows missing ['country_code', 'indicator_code'].
 cleaned_EdStatsFootNote.csv saved


VERIFICATIONS

In [ ]:

import pandas as pd

# Paths
original_file = "EdStatsSeries.csv"
cleaned_file = "cleaned_EdStatsSeries.csv"  # Ensure this exists after cleaning

# Loaded both files
original_df = pd.read_csv(original_file, low_memory=False)
cleaned_df = pd.read_csv(cleaned_file, low_memory=False)

# Compared column names
original_cols = set(original_df.columns)
cleaned_cols = set(cleaned_df.columns)

# Missing values count
original_missing = original_df.isnull().sum().sum()
cleaned_missing = cleaned_df.isnull().sum().sum()

# Duplicate rows count
original_duplicates = original_df.duplicated().sum()
cleaned_duplicates = cleaned_df.duplicated().sum()

# Summary of changes
summary = f"""
Verification Report for EdStatsSeries.csv
----------------------------------------
Column names changed: {original_cols - cleaned_cols if original_cols != cleaned_cols else 'No changes'}
Total columns before: {len(original_cols)}, after: {len(cleaned_cols)}

Missing values before cleaning: {original_missing}
Missing values after cleaning: {cleaned_missing}

Duplicate rows before cleaning: {original_duplicates}
Duplicate rows after cleaning: {cleaned_duplicates}

Rows before: {len(original_df)}, after: {len(cleaned_df)}
"""

# Save summary to a text file
with open("verification_summary.txt", "w") as f:
    f.write(summary)

print(summary)



Verification Report for EdStatsSeries.csv
----------------------------------------
Column names changed: {'Unnamed: 20', 'Aggregation method', 'Unit of measure', 'Related source links', 'Indicator Name', 'Source', 'Series Code', 'Base Period', 'Related indicators', 'License Type', 'Other web links', 'Long definition', 'General comments', 'Periodicity', 'Development relevance', 'Short definition', 'Other notes', 'Statistical concept and methodology', 'Limitations and exceptions', 'Topic', 'Notes from original source'}
Total columns before: 21, after: 15

Missing values before cleaning: 55203
Missing values after cleaning: 33213

Duplicate rows before cleaning: 0
Duplicate rows after cleaning: 0

Rows before: 3665, after: 3665



In [14]:

import pandas as pd

original_file = "EdStatsCountry.csv"
cleaned_file = "cleaned_EdStatsCountry.csv"

original_df = pd.read_csv(original_file, low_memory=False)
cleaned_df = pd.read_csv(cleaned_file, low_memory=False)

summary = f"""
Verification Report: EdStatsCountry.csv
---------------------------------------
Columns before: {len(original_df.columns)}, after: {len(cleaned_df.columns)}
Column name changes: {set(original_df.columns) - set(cleaned_df.columns) if set(original_df.columns) != set(cleaned_df.columns) else 'No changes'}

Missing values before: {original_df.isnull().sum().sum()}
Missing values after: {cleaned_df.isnull().sum().sum()}

Duplicate rows before: {original_df.duplicated().sum()}
Duplicate rows after: {cleaned_df.duplicated().sum()}

Rows before: {len(original_df)}, after: {len(cleaned_df)}
"""

with open("verify_EdStatsCountry.txt", "w") as f:
    f.write(summary)

print(summary)



Verification Report: EdStatsCountry.csv
---------------------------------------
Columns before: 32, after: 31
Column name changes: {'Alternative conversion factor', 'Balance of Payments Manual in use', 'SNA price valuation', 'Latest population census', 'Latest agricultural census', 'Latest water withdrawal data', 'Long Name', 'Source of most recent Income and expenditure data', 'Lending category', 'Region', 'Income Group', 'IMF data dissemination standard', 'Latest industrial data', 'Latest household survey', 'System of National Accounts', 'Special Notes', 'Other groups', 'External debt Reporting status', 'PPP survey year', 'Country Code', 'Latest trade data', 'National accounts reference year', 'Currency Unit', 'WB-2 code', 'System of trade', 'Vital registration complete', '2-alpha code', 'Table Name', 'Short Name', 'National accounts base year', 'Unnamed: 31', 'Government Accounting concept'}

Missing values before: 2354
Missing values after: 2113

Duplicate rows before: 0
Duplicate

In [15]:

import pandas as pd

original_file = "EdStatsCountry-Series.csv"
cleaned_file = "cleaned_EdStatsCountrySeries.csv"

original_df = pd.read_csv(original_file, low_memory=False)
cleaned_df = pd.read_csv(cleaned_file, low_memory=False)

summary = f"""
Verification Report: EdStatsCountry-Series.csv
----------------------------------------------
Columns before: {len(original_df.columns)}, after: {len(cleaned_df.columns)}
Column name changes: {set(original_df.columns) - set(cleaned_df.columns) if set(original_df.columns) != set(cleaned_df.columns) else 'No changes'}

Missing values before: {original_df.isnull().sum().sum()}
Missing values after: {cleaned_df.isnull().sum().sum()}

Duplicate rows before: {original_df.duplicated().sum()}
Duplicate rows after: {cleaned_df.duplicated().sum()}

Rows before: {len(original_df)}, after: {len(cleaned_df)}
"""

with open("verify_EdStatsCountrySeries.txt", "w") as f:
    f.write(summary)

print(summary)



Verification Report: EdStatsCountry-Series.csv
----------------------------------------------
Columns before: 4, after: 3
Column name changes: {'Unnamed: 3', 'CountryCode', 'DESCRIPTION', 'SeriesCode'}

Missing values before: 613
Missing values after: 0

Duplicate rows before: 0
Duplicate rows after: 0

Rows before: 613, after: 613



In [16]:

import pandas as pd

original_file = "EdStatsFootNote.csv"
cleaned_file = "cleaned_EdStatsFootNote.csv"

original_df = pd.read_csv(original_file, low_memory=False)
cleaned_df = pd.read_csv(cleaned_file, low_memory=False)

summary = f"""
Verification Report: EdStatsFootNote.csv
----------------------------------------
Columns before: {len(original_df.columns)}, after: {len(cleaned_df.columns)}
Column name changes: {set(original_df.columns) - set(cleaned_df.columns) if set(original_df.columns) != set(cleaned_df.columns) else 'No changes'}

Missing values before: {original_df.isnull().sum().sum()}
Missing values after: {cleaned_df.isnull().sum().sum()}

Duplicate rows before: {original_df.duplicated().sum()}
Duplicate rows after: {cleaned_df.duplicated().sum()}

Rows before: {len(original_df)}, after: {len(cleaned_df)}
"""

with open("verify_EdStatsFootNote.txt", "w") as f:
    f.write(summary)

print(summary)



Verification Report: EdStatsFootNote.csv
----------------------------------------
Columns before: 5, after: 4
Column name changes: {'SeriesCode', 'Unnamed: 4', 'Year', 'DESCRIPTION', 'CountryCode'}

Missing values before: 643638
Missing values after: 0

Duplicate rows before: 0
Duplicate rows after: 0

Rows before: 643638, after: 643638



In [ ]:

import pandas as pd


original_file = "EdStatsData.csv"
cleaned_file = "cleaned_EdStatsData.csv"


original_df = pd.read_csv(original_file, low_memory=False)
cleaned_df = pd.read_csv(cleaned_file, low_memory=False)


original_cols = set(original_df.columns)
cleaned_cols = set(cleaned_df.columns)


original_missing = original_df.isnull().sum().sum()
cleaned_missing = cleaned_df.isnull().sum().sum()

original_duplicates = original_df.duplicated().sum()
cleaned_duplicates = cleaned_df.duplicated().sum()

# Summary of changes
summary = f"""
Verification Report for EdStatsData.csv
----------------------------------------
Column names changed: {original_cols - cleaned_cols if original_cols != cleaned_cols else 'No changes'}
Total columns before: {len(original_cols)}, after: {len(cleaned_cols)}

Missing values before cleaning: {original_missing}
Missing values after cleaning: {cleaned_missing}

Duplicate rows before cleaning: {original_duplicates}
Duplicate rows after cleaning: {cleaned_duplicates}

Rows before: {len(original_df)}, after: {len(cleaned_df)}
"""

# Save summary to a text file
with open("verification_EdStatsData.txt", "w") as f:
    f.write(summary)

print(summary)



Verification Report for EdStatsData.csv
----------------------------------------
Column names changed: {'Country Code', 'Country Name', 'Indicator Name', 'Unnamed: 69', 'Indicator Code'}
Total columns before: 70, after: 69

Missing values before cleaning: 53455179
Missing values after cleaning: 0

Duplicate rows before cleaning: 0
Duplicate rows after cleaning: 0

Rows before: 886930, after: 357405

